##Funnel volumes to tables

In [0]:
%sql
USE CATALOG digital_funnel;
USE SCHEMA landing;
CREATE EXTERNAL VOLUME IF NOT EXISTS digital_funnel.landing.dataset_volume
LOCATION 'abfss://landing@firstprojectdlucext.dfs.core.windows.net/dataset';

####Converting dataset volumes in tables into bronze using SQL (Ingest raw data into bronze -still raw-)
1. usuarios_banco_con_funnel
2. clientes

In [0]:
%sql
---DROP TABLE IF EXISTS digital_funnel.bronze.clientes;
---DROP TABLE IF EXISTS digital_funnel.bronze.usuarios_banco_con_funnel;

In [0]:
def volume_to_table_dup(name, merge_condition):

  input_source_path = f'/Volumes/digital_funnel/landing/dataset_volume/{name}.csv'
  df = spark.read.format('csv').option('header', 'true').load(input_source_path)
  table_schema = df.schema
  table_schema = ", ".join([f"{field.name} {field.dataType.typeName()}" for field in table_schema])
  df.createOrReplaceTempView(f'{name}_increment')
  spark.sql(f"""CREATE TABLE IF NOT EXISTS digital_funnel.bronze.{name} ({table_schema})
                LOCATION 'abfss://bronze@firstprojectdlucext.dfs.core.windows.net/usuarios_banco_con_funnel/'""")

  spark.sql(f"""MERGE INTO digital_funnel.bronze.{name} AS tgt
                USING {name}_increment AS src
                ON ({merge_condition})
                WHEN NOT MATCHED THEN
                  INSERT *""")
  ###PARTITIONED BY (stage)

volume_to_table_dup("usuarios_banco_con_funnel", "tgt.ID_cliente = src.ID_cliente AND tgt.stage = src.stage") 

In [0]:
def volume_to_table(name, merge_condition):

  input_source_path = f'/Volumes/digital_funnel/landing/dataset_volume/{name}.csv'
  df = spark.read.format('csv').option('header', 'true').load(input_source_path)
  table_schema = df.schema
  table_schema = ", ".join([f"{field.name} {field.dataType.typeName()}" for field in table_schema])
  df.createOrReplaceTempView(f'{name}_increment')
  spark.sql(f"""CREATE TABLE IF NOT EXISTS digital_funnel.bronze.{name} ({table_schema})
                LOCATION 'abfss://bronze@firstprojectdlucext.dfs.core.windows.net/clientes/'""")

  spark.sql(f"""MERGE INTO digital_funnel.bronze.{name}  AS tgt
               USING {name}_increment AS src
               ON ({merge_condition})
               WHEN MATCHED THEN
                 UPDATE SET *
               WHEN NOT MATCHED THEN
                 INSERT *""")
  
volume_to_table("clientes", "tgt.ID_cliente = src.ID_cliente") 

In [0]:
%sql
SELECT * FROM digital_funnel.bronze.usuarios_banco_con_funnel;

In [0]:
##table_names = []
##for file_info in dbutils.fs.ls('/Volumes/digital_funnel/landing/dataset_volume'):
##    table_name = file_info.name.split(".")[0].rstrip('/')
##    table_names.append(table_name)
    
##for i in table_names:
##    dbutils.fs.mv("/Volumes/digital_funnel/landing/dataset_volume/{name}.csv", "abfss://bronze@firstprojectdlucext.dfs.core.windows.net/old_files/"